<a href="https://colab.research.google.com/github/Tdesius/ttnt/blob/main/lec04_uninformedsearch_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Students refer to pseudo codes of BFS, DFS, UCS, DLS, and IDS in [this link](https://drive.google.com/file/d/1q2LtrRCfemfiqyhfxNMcVJ3alvLh_pdV/view?usp=share_link) to implement the corresponding classes in TODO 1 - 5. \
Students can add supporting attributes and methods to the five classes of search strategies as needed.

# Libraries

In [90]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [91]:
import os
import heapq

# Graph class

In [92]:
# Directed, weighted graphs
class Graph:
  def __init__(self):
    self.AL = dict() # adjacency list
    self.V = 0
    self.E = 0

  def __str__(self):
    res = 'V: %d, E: %d\n'%(self.V, self.E)
    for u, neighbors in self.AL.items():
      line = '%d: %s\n'%(u, str(neighbors))
      res += line
    return res

  def print(self):
    print(str(self))

  def load_from_file(self, filename):
    '''
        Example input file:
            V E
            u v w
            u v w
            u v w
            ...

        # input.txt
        7 8
        0 1 5
        0 2 6
        1 3 12
        1 4 9
        2 5 5
        3 5 8
        3 6 7
        4 6 4
    '''
    if os.path.exists(filename):
      with open(filename) as g:
        self.V, self.E = [int(it) for it in g.readline().split()]
        for line in g:
          u, v, w = [int(it) for it in line.strip().split()]
          if u not in self.AL:
            self.AL[u] = []
          self.AL[u].append((v, w))

In [93]:
g = Graph()
g.load_from_file('/content/drive/MyDrive/input.txt')
g.print()

V: 7, E: 8
0: [(1, 5), (2, 6)]
1: [(3, 12), (4, 9)]
2: [(5, 5)]
3: [(5, 8), (6, 7)]
4: [(6, 4)]



# Search Strategies

In [94]:
class SearchStrategy:
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    return expanded, path

  def reconstruct_path(self, dst, parent):
    path = []
    if dst in parent:
        while dst is not None:
            path.insert(0, dst)
            dst = parent[dst]
    return path

In [95]:
class BFS(SearchStrategy):
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    frontier = [src]
    explored = set()
    parent = {src: None}
    # TODO 1
    while frontier:
      node = frontier.pop(0)
      expanded.append(node)
      if node == dst:
        break
      explored.add(node)
      for neighbor, _ in g.AL.get(node, []):
        if neighbor not in explored and neighbor not in frontier:
          parent[neighbor] = node
          frontier.append(neighbor)

    return expanded, self.reconstruct_path(dst, parent)

In [96]:
class DFS(SearchStrategy):
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    frontier =[src]
    explored = set()
    parent = {src: None}
    # TODO 2
    while frontier:
      node = frontier.pop()
      if node in explored:
        continue
      expanded.append(node)
      explored.add(node)
      if node == dst:
        break
      for neighbor, _ in reversed(g.AL.get(node, [])):
        if neighbor not in explored:
          parent[neighbor] = node
          frontier.append(neighbor)

    return expanded, self.reconstruct_path(dst, parent)

In [97]:
class UCS(SearchStrategy):
  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    frontier = [(0, src)]
    explored = set()
    parent = {src: None}
    # TODO 3
    while frontier:
      cost, node = heapq.heappop(frontier)
      if node in explored:
        continue
        expanded.append(node)
        explored[node] = cost
        if node == dst:
          break
        for neighbor, weight in g.AL.get(node, []):
          new_cost = cost + weight
          if neighbor not in explored or new_cost < explored[neighbor]:
            parent[neighbor] = node
            heapq.heappush(froniter, (new_cost,neighbor))

    return expanded, self.reconstruct_path(dst, parent)

In [98]:
class DLS(SearchStrategy):
  def __init__(self, LIM: int):
    self.LIM = LIM

  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst
    parent = {src: None}
    # TODO 4
      def recursive_dls(node, depth, parent):
        if depth > self.LIM:
          return False
        expanded.append(node)
        if node == dst:
          return True
        for neighbor, _ in g.AL.get(node, []):
          if neighbor not in parent:
            parent[neighbor] = node
            if recursive_dls(neighbor, depth + 1, parent):
              return True
        return False

    if recursive_dls(src, 0, parent):
      path = self.reconstruct_path(dst, parent)

    return expanded, path

In [99]:
class IDS(SearchStrategy):
  def __init__(self, MAX_LIM: int):
    self.MAX_LIM = MAX_LIM

  def search(self, g: Graph, src: int, dst: int) -> tuple:
    expanded = [] # list of expanded vertices in the traversal order
    path = [] # path from src to dst

    # TODO 5
    for depth in range(self.MAX_LIM + 1):
      dls = DLS(depth)
      expanded, path = dls.search(g, src, dst)
      if path:
        return expanded, path
    return [], []

# Evaluation

In [100]:
bfs = BFS()
dfs = DFS()
ucs = UCS()
dls = DLS(LIM=3)
ids = IDS(MAX_LIM=5)

for stg in [bfs, dfs, ucs, dls, ids]:
  print(stg)
  expanded, path = stg.search(g, 0, g.V-1)
  print(expanded)
  print(path)




[0, 1, 2, 3, 4, 5, 6]
[0, 1, 3, 6]
[0, 1, 3, 5, 6]
[0, 1, 3, 6]
[]
[]
[]
[]
[]
[]


# Submission

*   Students download the notebook after completion
*   Rename the notebook in which inserting your student ID at the beginning. \
For example, **123456-lec04-UninformedSearch-HW.ipynb**
*   Finally, submit the file

